In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from bs4 import BeautifulSoup
import lxml
import requests
import unicodedata

In [2]:
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))
warnings.simplefilter(action='ignore', category=(FutureWarning))

In [3]:
fm_df = pd.read_csv("players_db/fm23/uluta_fm23data.csv")

In [4]:
fm_df.columns = [col.strip() for col in fm_df.columns]

for col in fm_df.select_dtypes("object").columns:
   fm_df[col] = fm_df[col].apply(lambda x: x.strip())

In [5]:
# Assume df is your DataFrame
new_column_names = {
    'Acc': 'Acceleration',
    'Aer': 'Aerial_Reach',
    'Agg': 'Aggression',
    'Agi': 'Agility',
    'Ant': 'Anticipation',
    'Bal': 'Balance',
    'Bra': 'Bravery',
    'Cmd': 'Command_of_Area',
    'Com': 'Communication',
    'Cmp': 'Composure',
    'Cnt': 'Concentration',
    'Cor': 'Corners',
    'Cro': 'Crossing',
    'Dec': 'Decisions',
    'Det': 'Determination',
    'Dri': 'Dribbling',
    'Ecc': 'Eccentricity',
    'Fin': 'Finishing',
    'Fir': 'First_Touch',
    'Fla': 'Flair',
    'Fre': 'Free_Kick_Taking',
    'Han': 'Handling',
    'Hea': 'Heading',
    'Jum': 'Jumping_Reach',
    'Kic': 'Kicking',
    'Ldr': 'Leadership',
    'Lon': 'Long_Shots',
    'L Th': 'Long_Throws',
    'Mar': 'Marking',
    'Nat .1': 'Natural_Fitness',
    'OtB': 'Off_the_Ball',
    '1v1': 'One_on_Ones',
    'Pac': 'Pace',
    'Pas': 'Passing',
    'Pen': 'Penalty_Taking',
    'Pos': 'Positioning',
    'Pun': 'Punching',
    'Ref': 'Reflexes',
    'TRO': 'Rushing_Out',
    'Sta': 'Stamina',
    'Str': 'Strength',
    'Tck': 'Tackling',
    'Tea': 'Teamwork',
    'Tec': 'Technique',
    'Thr': 'Throwing',
    'Vis': 'Vision',
    'Wor': 'Work_Rate'
}

# new_column_names = {key: value.lower() for key, value in new_column_names.items()}
fm_df.rename(columns=new_column_names, inplace=True)

In [6]:
fm_df.query("Name == 'Joshua Kimmich'").iloc[0].to_dict()

{'UID': 92039023,
 'Inf': '',
 'Name': 'Joshua Kimmich',
 'DoB': '8/2/1995 (27 years old)',
 'Nat': 'GER',
 'Division': 'Bundesliga',
 'Club': 'FC Bayern',
 'Based': 'Germany (Bundesliga)',
 'Preferred Foot': 'Right',
 'Right Foot': 'Very Strong',
 'Left Foot': 'Fairly Strong',
 'Position': 'D/WB (R), DM, M (C)',
 'Height': '177 cm',
 'Weight': '75 kg',
 'Age': 27,
 'Transfer Value': '€96M - €119M',
 'Wage': '€1,496,000 p/m',
 'AT Apps': '253',
 'AT Gls': '23',
 'Team': '',
 'Caps': 68,
 'Yth Apps': '14',
 'Style': 'Leader',
 'Rc Injury': '-',
 'Best Role': 'Deep Lying Playmaker',
 'Best Duty': 'Defend',
 'Best Pos': 'DM',
 'Acceleration': 13,
 'Aerial_Reach': 1,
 'Aggression': 15,
 'Agility': 12,
 'Anticipation': 15,
 'Balance': 13,
 'Bravery': 15,
 'Command_of_Area': 2,
 'Communication': 2,
 'Composure': 17,
 'Concentration': 14,
 'Corners': 15,
 'Crossing': 17,
 'Decisions': 16,
 'Determination': 20,
 'Dribbling': 12,
 'Eccentricity': 3,
 'Finishing': 11,
 'First_Touch': 15,
 'Flair

In [7]:
gk_attributes = [ "Aerial_Reach","Command_of_Area", "Communication","Eccentricity", "First_Touch", "Handling", "Kicking", "One_on_Ones", "Punching", "Reflexes", "Rushing_Out", "Throwing", "Passing" ]
fm_attributes = list(new_column_names.values())

fm_df[fm_attributes] = (fm_df[fm_attributes] * 4.95).astype(int)

In [8]:
fm_df.head(n=5)

,UID,Inf,Name,DoB,Nat,Division,Club,Based,Preferred Foot,Right Foot,...,Reflexes,Rushing_Out,Stamina,Strength,Tackling,Teamwork,Technique,Throwing,Vision,Work_Rate
0,831079,,Giorgio Chiellini,14/8/1984 (37 years old),ITA,Major League Soccer,LAFC,U.S.A. (MLS),Left Only,Weak,...,4,9,49,69,89,84,39,19,34,74
1,2105510,,Hulk,25/7/1986 (35 years old),BRA,Brazilian National First Division,ATM,Brazil (First Division),Left,Reasonable,...,9,4,74,89,39,39,74,14,69,54
2,78000335,,Luis Suárez,24/1/1987 (35 years old),URU,Uruguayan First Division,Nacional,Uruguay (First Division),Right,Very Strong,...,4,14,49,69,44,64,74,14,74,44
3,962988,Ret,Gonzalo Higuaín,10/12/1987 (34 years old),ARG,Major League Soccer,Inter Miami,U.S.A. (MLS),Right,Very Strong,...,4,14,49,79,39,49,79,4,69,49
4,19007730,,Alex Teixeira,6/1/1990 (32 years old),BRA,Brazilian National Second Division,VDG,Brazil (Second Division),Right,Very Strong,...,4,14,54,34,24,49,79,9,54,49


In [9]:
fm_df['DoB'][0]

'14/8/1984 (37 years old)'

In [10]:
convert_date_format = lambda x: pd.to_datetime(x, format='%d/%m/%Y').strftime('%Y-%m-%d')
fm_df['DoB'] = fm_df['DoB'].apply(lambda x: x.split(" ")[0])
fm_df['DoB'] = fm_df['DoB'].apply(convert_date_format)

In [11]:
fm_df['DoB'][0]

'1984-08-14'

In [12]:
fm_df['Height'] = fm_df['Height'].apply(lambda x: x.split(" ")[0])

In [13]:
fm_df.columns

Index(['UID', 'Inf', 'Name', 'DoB', 'Nat', 'Division', 'Club', 'Based',
       'Preferred Foot', 'Right Foot', 'Left Foot', 'Position', 'Height',
       'Weight', 'Age', 'Transfer Value', 'Wage', 'AT Apps', 'AT Gls', 'Team',
       'Caps', 'Yth Apps', 'Style', 'Rc Injury', 'Best Role', 'Best Duty',
       'Best Pos', 'Acceleration', 'Aerial_Reach', 'Aggression', 'Agility',
       'Anticipation', 'Balance', 'Bravery', 'Command_of_Area',
       'Communication', 'Composure', 'Concentration', 'Corners', 'Crossing',
       'Decisions', 'Determination', 'Dribbling', 'Eccentricity', 'Finishing',
       'First_Touch', 'Flair', 'Free_Kick_Taking', 'Handling', 'Heading',
       'Jumping_Reach', 'Kicking', 'Leadership', 'Long_Shots', 'Long_Throws',
       'Marking', 'Natural_Fitness', 'Off_the_Ball', 'One_on_Ones', 'Pace',
       'Passing', 'Penalty_Taking', 'Positioning', 'Punching', 'Reflexes',
       'Rushing_Out', 'Stamina', 'Strength', 'Tackling', 'Teamwork',
       'Technique', 'Throwing', 

In [14]:
fm_df['Club_id'] = pd.Categorical(fm_df['Club']).codes
fm_df['League_id'] = pd.Categorical(fm_df['Division']).codes

In [15]:
fm_df = fm_df[fm_df['Inf'] != 'Yth']

In [16]:
fm_df = fm_df.rename(columns={"Best Pos": "Best_Pos"})

<br><br><br><br>
# Merged_DF

In [17]:
selected_leagues = ['English Premier Division',
 'Ligue 1 Uber Eats',
 'Spanish First Division',
 'Bundesliga',
 'Italian Serie A',
 'Eredivisie',
 'Portuguese Premier League',
 'Turkish Super League']

In [18]:
fm_df = fm_df[fm_df.Division.isin(selected_leagues)]

In [19]:
fm_df.columns

Index(['UID', 'Inf', 'Name', 'DoB', 'Nat', 'Division', 'Club', 'Based',
       'Preferred Foot', 'Right Foot', 'Left Foot', 'Position', 'Height',
       'Weight', 'Age', 'Transfer Value', 'Wage', 'AT Apps', 'AT Gls', 'Team',
       'Caps', 'Yth Apps', 'Style', 'Rc Injury', 'Best Role', 'Best Duty',
       'Best_Pos', 'Acceleration', 'Aerial_Reach', 'Aggression', 'Agility',
       'Anticipation', 'Balance', 'Bravery', 'Command_of_Area',
       'Communication', 'Composure', 'Concentration', 'Corners', 'Crossing',
       'Decisions', 'Determination', 'Dribbling', 'Eccentricity', 'Finishing',
       'First_Touch', 'Flair', 'Free_Kick_Taking', 'Handling', 'Heading',
       'Jumping_Reach', 'Kicking', 'Leadership', 'Long_Shots', 'Long_Throws',
       'Marking', 'Natural_Fitness', 'Off_the_Ball', 'One_on_Ones', 'Pace',
       'Passing', 'Penalty_Taking', 'Positioning', 'Punching', 'Reflexes',
       'Rushing_Out', 'Stamina', 'Strength', 'Tackling', 'Teamwork',
       'Technique', 'Throwing', 

In [20]:
for leg in selected_leagues:
    q = fm_df.query(f"Division == '{leg}'")["Club"].value_counts()
    print(leg , "------", len(q.to_list()))

English Premier Division ------ 20
Ligue 1 Uber Eats ------ 20
Spanish First Division ------ 20
Bundesliga ------ 18
Italian Serie A ------ 20
Eredivisie ------ 18
Portuguese Premier League ------ 18
Turkish Super League ------ 19


In [21]:
fm_df.query(f"Division == 'Turkish Super League'")["Club"].value_counts()

Club
Kayserispor         41
Beşiktaş            41
Trabzonspor         40
Adana D.S.          39
Fenerbahçe          37
Ankaragücü          37
İstanbulspor        36
Galatasaray         35
Hatayspor           35
Kasımpaşa           35
Antalyaspor         34
Alanyaspor          34
Başakşehir FK       34
Sivasspor           33
Giresunspor         33
Gaziantep FK        32
Fatih Karagümrük    32
Ümraniyespor        32
Konyaspor           31
Name: count, dtype: int64

In [22]:
# fm_df.query(f"Club == 'Adana D.S.'").head()

<br><br><br><br>
# Team Names Converter from fminside.com

In [23]:
url_df = fm_df.drop_duplicates(subset=["Club"], keep="first")
url_df.loc[:, 'UID'] = url_df['UID'].apply(lambda text: str(text))
url_df = url_df[["Club", "UID"]]

In [24]:
def Player_to_URL(player_id):
    slug = player_id + "-x-x"
    return f"https://fminside.net/players/3-fm-23/{slug}"

In [25]:
def Convert_Team_Name_From_URL(player_id, club):
    url = Player_to_URL(player_id)
    req = requests.get(url)
    html = req.content
    soup = BeautifulSoup(html, "lxml")
    club_name_element = soup.find('div', class_='meta').find('ul').find_all('li')[0].text
    club_name = club_name_element.strip()
    return {"fm":club,"normal":club_name}

In [26]:
team_mapping_array = []
raw_names = url_df.to_dict()
uids = url_df["UID"]
clubs = url_df["Club"]
clubs_array = clubs.to_list()

In [27]:
# for uid,club in zip(uids,clubs):
#     new_entry = Convert_Team_Name_From_URL(uid, club)
#     print(new_entry)
#     team_mapping_array.append(new_entry)

In [28]:
fm_df.query("Name == 'Joshua Kimmich'")

,UID,Inf,Name,DoB,Nat,Division,Club,Based,Preferred Foot,Right Foot,...,Stamina,Strength,Tackling,Teamwork,Technique,Throwing,Vision,Work_Rate,Club_id,League_id
42332,92039023,,Joshua Kimmich,1995-02-08,GER,Bundesliga,FC Bayern,Germany (Bundesliga),Right,Very Strong,...,99,54,59,84,79,4,89,94,3704,185


In [29]:
fminside_normal_names = pd.read_csv("players_db/fm23/fminside_team_names.csv")
# merged_df = pd.merge(fm_df, fminside_normal_names, how='inner', left_on='Club', right_on='fm')
# merged_df['Club'] = merged_df['normal'].where(merged_df['Club'] == merged_df['fm'], merged_df['Club'])
# # merged_df.drop(['fm', 'normal'], axis=1, inplace=True)
# merged_df = merged_df[merged_df.fm.isin(clubs_array)]

In [30]:
fminside_club_names = []
for index, row in fminside_normal_names.iterrows():
  x_fm = row["fm"]
  x_normal = row["normal"]
  club_dict = {"fm": x_fm, "normal": x_normal}
  fminside_club_names.append(club_dict)

In [31]:
def Rename_Club(club_name):
    matched_club = ""
    for f in fminside_club_names:
        if f["fm"] == club_name:
            return f["normal"]

In [32]:
fm_df['Club'] = fm_df["Club"].apply(lambda x: Rename_Club(x))

In [33]:
fm_df.query("Name == 'Joshua Kimmich'")

,UID,Inf,Name,DoB,Nat,Division,Club,Based,Preferred Foot,Right Foot,...,Stamina,Strength,Tackling,Teamwork,Technique,Throwing,Vision,Work_Rate,Club_id,League_id
42332,92039023,,Joshua Kimmich,1995-02-08,GER,Bundesliga,FC Bayern,Germany (Bundesliga),Right,Very Strong,...,99,54,59,84,79,4,89,94,3704,185


<br><br><br><br>
# Position Styling

In [34]:
position_mapping = {
    'M (C)': 'MC',
    'D (C)': 'DC',
    'GK': 'GK',
    'ST (C)': 'ST',
    'AM (L)': 'AML',
    'AM (R)': 'AMR',
    'D (R)': 'DR',
    'D (L)': 'DL',
    'DM': 'DM',
    'AM (C)': 'AMC',
    'M (L)': 'ML',
    'M (R)': 'MR',
    'WB (L)': 'WBL',
    'WB (R)': 'WBR'
}

In [35]:
def remove_parentheses(text):
    return text.replace("(", "").replace(")", "")

In [36]:
def pos_map(position):
    return position_mapping[position]

In [37]:
fm_df['Best_Pos'].value_counts()

Best_Pos
M (C)     977
D (C)     938
AM (L)    738
GK        704
ST (C)    648
AM (R)    555
D (R)     521
D (L)     475
DM        388
AM (C)    180
M (L)     130
M (R)     116
WB (R)     29
WB (L)     23
Name: count, dtype: int64

In [38]:
fm_df['Position'].value_counts()

Position
D (C)                    749
GK                       704
DM, M (C)                600
ST (C)                   519
M/AM (C)                 279
                        ... 
D (R), WB/M (RL)           1
D (R), WB (RL), M (R)      1
D (RLC), WB/M (R)          1
WB/M (RL)                  1
D/AM (R)                   1
Name: count, Length: 224, dtype: int64

In [39]:
def Multiple_Position_Beautifier(position_value):
    x = ""
    positions = position_value.split(",")
    for p in positions:
        p = p.strip()
        if p in position_mapping.keys():
            x += f"{position_mapping[p]},"
        elif "/" in p:
            slashed = p.split(" ")
            locations = slashed[0].split("/")
            sides = list(remove_parentheses(slashed[1]))
            for loc in locations:
                for side in sides:
                    splitted_pos = loc+side
                    x += f"{splitted_pos},"
        else:
            slashed = p.split(" ")
            loc = slashed[0]
            sides = list(remove_parentheses(slashed[1]))
            for side in sides:
                splitted_pos = loc+side
                x += f"{splitted_pos},"
    return x[:-1]

In [40]:
fm_df['Position'] = fm_df['Position'].apply(lambda row: Multiple_Position_Beautifier(row))
fm_df['Best_Pos'] = fm_df['Best_Pos'].apply(lambda row: Multiple_Position_Beautifier(row))

In [41]:
position_order = ['GK', 'WBR', 'DR', 'DC', 'DL', 'WBL', 'DM',  
                  'MC', 'MR', 'AMC', 'ML', 'AMR', 'ST', 'AML']
def get_position_rank(position):
    return position_order.index(position) if position in position_order else len(position_order)
fm_df['Pos_Rank'] = fm_df['Best_Pos'].apply(get_position_rank)

In [42]:
fm_df.query("Club == 'Chaves'")[["Best_Pos"]]

,Best_Pos
168395,DC
168409,DC
168573,AML
168584,ST
168616,AML
168825,AML
169038,AMR
169074,GK
169161,MC
169202,ML


In [43]:
fm_df['Position'].value_counts()

Position
DC                  749
GK                  704
DM,MC               600
ST                  519
MC,AMC              279
                   ... 
DR,WBR,WBL,MR,ML      1
DR,WBR,WBL,MR         1
DR,DL,DC,WBR,MR       1
WBR,WBL,MR,ML         1
DR,AMR                1
Name: count, Length: 224, dtype: int64

In [44]:
fm_df['Best_Pos'].value_counts()

Best_Pos
MC     977
DC     938
AML    738
GK     704
ST     648
AMR    555
DR     521
DL     475
DM     388
AMC    180
ML     130
MR     116
WBR     29
WBL     23
Name: count, dtype: int64

<br><br><br><br>
# Players' Overall

- Keeper -> gk
- Defending -> def 
- Passing -> pas
- Dribbling -> dri
- Finishing -> fin
- Stamina -> sta
- Strength -> str
- Heading -> hed
- Mental -> men
- Intelligence -> iq

In [45]:
rating_attrs = ["gk","def","pas","dri","fin","sta","str","hed","men","iq"]

In [46]:
def Dict_Generator(attrs):
    weights = (len(attrs)*"0.1,").split(",")[:-1]
    return {key: float(weights[i]) for i, key in enumerate(attrs)}

<h3 style="color:orange;">  Keeper</h3>

In [47]:
current_ability = "gk"
fm_df[current_ability] = fm_df[gk_attributes].mean(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

,Name,gk
42335,Manuel Neuer,75
167966,Marc-André ter Stegen,74
151645,Ederson,72
151647,Alisson,72
167963,Jan Oblak,69


<h3 style="color:orange;">  Defending</h3>

In [48]:
weights =   {'Positioning': 0.35,
             'Heading': 0.05,
             'Marking': 0.30,
             'Tackling': 0.35}
expert_attributes = list(weights.keys())

current_ability = "def"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

,Name,def
151660,Thiago Silva,89
151642,Rúben Dias,89
151644,Virgil van Dijk,88
73611,Chris Smalling,88
168028,Sebastián Coates,87


<h3 style="color:red;">  Passing</h3>

In [49]:
weights =   {'Crossing': 0.10,
             'First_Touch': 0.10,
             'Passing': 0.35,
             'Technique': 0.20,
             'Decisions': 0.05,
             'Determination': 0.05,
             'Vision': 0.15}
expert_attributes = list(weights.keys())

current_ability = "pas"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

,Name,pas
167956,Lionel Messi,93
167959,Luka Modrić,92
151638,Kevin De Bruyne,89
167960,Karim Benzema,89
167970,Toni Kroos,88


<h3 style="color:red;">  Dribbling</h3>

In [50]:
weights =   {'Dribbling': 0.35,
             'First_Touch': 0.05,
             'Acceleration': 0.20,
             'Agility': 0.10,
             'Balance': 0.10,
             'Natural_Fitness': 0.05,
             'Pace': 0.15}
expert_attributes = list(weights.keys())

current_ability = "dri"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

,Name,dri
167957,Kylian Mbappé,89
151738,Adama Traoré,86
167956,Lionel Messi,85
151640,Mohamed Salah,85
167964,Vinícius Júnior,84


<h3 style="color:red;">  Finishing</h3>

In [51]:
weights =   {'Finishing': 1}
expert_attributes = list(weights.keys())

current_ability = "fin"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

,Name,fin
151646,Cristiano Ronaldo,94
73550,Zlatan Ibrahimović,94
167958,Robert Lewandowski,94
151641,Harry Kane,94
73547,Ciro Immobile,89


<h3 style="color:green;">  Stamina</h3>

In [52]:
weights =   {'Stamina': 1}
expert_attributes = list(weights.keys())

current_ability = "sta"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

,Name,sta
42332,Joshua Kimmich,99
151729,James Ward-Prowse,99
73553,Marcelo Brozović,99
151691,Mason Mount,99
151758,Pierre-Emile Højbjerg,99


<h3 style="color:green;">  Strength</h3>

In [53]:
weights =   {'Strength': 1}
expert_attributes = list(weights.keys())

current_ability = "str"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

,Name,str
151713,Kurt Zouma,99
73542,Romelu Lukaku,99
152008,Fraser Forster,94
151685,Reece James,94
151812,Aleksandar Mitrović,94


<h3 style="color:green;">  Heading</h3>

In [54]:
weights =   {'Heading': 1}
expert_attributes = list(weights.keys())

current_ability = "hed"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

,Name,hed
167965,Sergio Ramos,99
73561,Olivier Giroud,94
75806,Milan Đurić,94
151646,Cristiano Ronaldo,94
167994,Edinson Cavani,89


<h3 style="color:purple;">  Mental</h3>

In [55]:
weights =   {'Aggression': 0.15,
             'Composure': 0.40,
             'Concentration': 0.45}
expert_attributes = list(weights.keys())

current_ability = "men"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

,Name,men
151660,Thiago Silva,90
167993,Axel Witsel,86
151645,Ederson,84
167980,Sergio Busquets,84
167959,Luka Modrić,84


<h3 style="color:purple;">  Intelligence</h3>

In [56]:
weights =   {'Vision': 0.35,
             'Anticipation': 0.20,
             'Decisions': 0.25,
             'Flair': 0.20}
expert_attributes = list(weights.keys())

current_ability = "iq"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

,Name,iq
167956,Lionel Messi,93
151638,Kevin De Bruyne,86
105366,Mesut Özil,85
168016,Pedri,85
167988,Marco Verratti,84


<h1 style="color:red;">  🥽TPR</h1>

In [57]:
positions = fm_df['Best_Pos'].unique().tolist()

In [58]:
positional_rating_weights = pd.read_csv('players_db/fm23/positional_rating_weights.csv',index_col=0)

In [59]:
positional_rating_weights

,DL,DC,DR,WBL,WBR,DM,ML,MC,MR,AML,AMC,AMR,ST,GK
Aggression,0.028048,0.029816,0.029955,0.022472,0.030588,0.027013,0.027047,0.025880,0.025847,0.000000,0.024658,0.024424,0.027769,0.000000
Anticipation,0.030303,0.054307,0.030360,0.026966,0.028235,0.055584,0.032164,0.030952,0.031343,0.030790,0.031507,0.032601,0.048701,0.000000
Bravery,0.029598,0.053562,0.029821,0.022472,0.025882,0.027844,0.026316,0.028468,0.025104,0.028112,0.028082,0.025486,0.030504,0.000000
Composure,0.029457,0.031626,0.028876,0.030337,0.025882,0.046338,0.031433,0.029710,0.030154,0.029005,0.054110,0.031008,0.031976,0.000000
Concentration,0.028048,0.031520,0.028471,0.026966,0.023529,0.029610,0.023392,0.027433,0.026441,0.026328,0.027397,0.027185,0.028716,0.000000
Decisions,0.029035,0.031839,0.029686,0.029213,0.028235,0.031169,0.029240,0.030435,0.027629,0.027220,0.032192,0.031114,0.030083,0.000000
Determination,0.052149,0.055053,0.053029,0.033708,0.032941,0.030857,0.028509,0.049068,0.030749,0.034360,0.030137,0.045768,0.056800,0.000000
Flair,0.026638,0.022042,0.024828,0.052809,0.061176,0.025766,0.054825,0.051553,0.046940,0.060241,0.028767,0.057980,0.030293,0.000000
Leadership,0.022833,0.033010,0.000000,0.000000,0.023529,0.025558,0.000000,0.026501,0.000000,0.026328,0.000000,0.000000,0.026612,0.000000
Off_the_Ball,0.050599,0.020552,0.027527,0.033708,0.028235,0.027948,0.059942,0.031159,0.051099,0.033467,0.052055,0.054157,0.051015,0.000000


In [60]:
def TPR_by_Position(row, pos=False):
    position = row["Best_Pos"] 
    rating_weights = positional_rating_weights["ST"].to_dict()
    rating_attrs = list(rating_weights.keys())
    tpr = (row[rating_attrs] * pd.Series(rating_weights)).sum()
    return int(tpr)

# Positional Ratings

In [61]:
posdf = pd.read_csv("players_db/fm23/position_regression_formulas.csv", index_col=0)
posdf

,GK,WBR,DR,DC,DL,WBL,DM,MC,MR,AMC,ML,AMR,ST,AML
GK,19.12,1.00,1.09,1.04,1.09,1.00,1.00,1.04,1.00,1.18,1.00,1.07,1.13,1.00
WBR,1.14,19.67,15.97,2.35,2.50,2.00,2.55,2.35,14.07,1.31,4.03,3.24,1.22,2.32
DR,1.22,17.54,19.13,5.73,3.15,2.00,3.57,2.29,12.19,1.08,3.19,2.26,1.15,1.67
DC,1.26,3.04,9.15,19.75,8.58,3.44,8.59,2.66,2.40,1.00,1.83,1.38,1.25,1.04
DL,1.13,9.04,8.81,4.84,19.02,17.94,2.22,1.86,5.26,1.30,9.14,2.09,1.00,1.78
WBL,1.06,8.96,6.73,2.22,16.05,19.62,1.91,1.84,4.88,1.31,10.51,2.68,1.04,2.55
DM,1.15,3.50,4.29,6.10,3.55,3.75,19.36,14.61,3.00,5.36,2.97,1.98,1.23,1.76
MC,1.33,4.46,3.72,2.77,3.12,6.94,17.06,19.64,6.07,15.31,8.02,6.44,1.89,6.50
MR,1.23,13.46,8.68,1.21,2.04,2.19,3.48,5.77,17.79,7.09,10.46,11.39,2.78,9.60
AMC,1.38,3.21,1.64,1.19,1.57,3.06,4.87,11.77,6.17,19.50,10.40,12.42,7.41,13.28


In [62]:
fm_df["tpr"] = fm_df.apply(TPR_by_Position, axis=1)

In [63]:
def Position_Regression(player_row, target_position):
    best_position = player_row.Best_Pos
    player_positions = player_row.Position
    tpr_value = player_row['tpr']
    
    coefficient = posdf.loc[best_position, target_position]
    base_value = int(coefficient / 20 * tpr_value)

    if target_position in player_positions.split(","):
        posdf_value = tpr_value - int((tpr_value - base_value) * 0.15)
        tpr_by_position_value = tpr_value - int((tpr_value - base_value) * 0.15)
        regression_value = max(posdf_value, tpr_by_position_value)
    else:
        regression_value = base_value

    return int(regression_value)

In [64]:
for pos in positions:
    fm_df[f"tpr_{pos}"] = fm_df.apply(
        lambda row: Position_Regression(row, pos), axis=1
    )

In [65]:
tpr_cols = []
for col in fm_df.columns:
    if "tpr_" in col:
        tpr_cols.append(col)
tpr_cols

['tpr_DM',
 'tpr_AML',
 'tpr_AMC',
 'tpr_GK',
 'tpr_MC',
 'tpr_DC',
 'tpr_AMR',
 'tpr_DL',
 'tpr_DR',
 'tpr_ST',
 'tpr_ML',
 'tpr_MR',
 'tpr_WBR',
 'tpr_WBL']

In [66]:
fm_df[["tpr","Best_Pos"]]

,tpr,Best_Pos
42332,72,DM
42333,74,AML
42334,75,AMC
42335,68,GK
42336,74,MC
...,...,...
189256,39,AMR
189257,28,GK
189261,35,AMR
189267,29,DC


In [67]:
# Normalize tprs
def Normalize(X, min_val, max_val):
    min_col = X.min()
    max_col = X.max()
    standardized_value = (X - min_col) / (max_col - min_col) * (max_val - min_val) + min_val
    return standardized_value.astype(int)

for tpr_column in tpr_cols:
    fm_df[tpr_column] = Normalize(fm_df[tpr_column], 45, 99)
    
fm_df["tpr"] = Normalize(fm_df["tpr"], 45, 95)

for pos in fm_df['Best_Pos'].unique():
    group = fm_df[fm_df['Best_Pos'] == pos]
    min_tpr = group['tpr'].min()
    max_tpr = group['tpr'].max()
    # Min-Max Normalization
    fm_df.loc[fm_df['Best_Pos'] == pos, 'tpr_normalized'] = Normalize(group['tpr'], 45, 94)

fm_df["tpr_GK"] = fm_df.apply(lambda row: int(row["tpr_GK"]/2) if row.Best_Pos != "GK" else row["tpr_GK"], axis=1)

In [68]:
fm_df.sort_values("tpr",ascending=False)[["Club","Name","Best_Pos","tpr","tpr_normalized"]].head(40)

,Club,Name,Best_Pos,tpr,tpr_normalized
167958,FC Barcelona,Robert Lewandowski,ST,95,94.0
151639,Manchester City,Erling Haaland,ST,93,91.0
73550,AC Milan,Zlatan Ibrahimović,ST,92,90.0
151641,Tottenham Hotspur,Harry Kane,ST,91,89.0
151640,Liverpool,Mohamed Salah,AMR,90,94.0
42334,FC Bayern,Thomas Müller,AMC,90,94.0
73542,Internazionale,Romelu Lukaku,ST,90,88.0
151659,Manchester United,Bruno Fernandes,AMC,90,94.0
151643,Manchester City,Bernardo Silva,AMR,90,94.0
73547,Lazio,Ciro Immobile,ST,90,88.0


In [69]:
ex_row = fm_df.query("Name in 'Jude Bellingham'").iloc[0]
ex_row[["tpr"] + tpr_cols]

tpr        86
tpr_DM     96
tpr_AML    61
tpr_AMC    83
tpr_GK     23
tpr_MC     96
tpr_DC     51
tpr_AMR    60
tpr_DL     52
tpr_DR     54
tpr_ST     48
tpr_ML     66
tpr_MR     60
tpr_WBR    56
tpr_WBL    63
Name: 42348, dtype: object

# Matching fbref Player Names with FM Player Names

In [70]:
from rapidfuzz import process
def find_best_match(name, choices):
    return process.extractOne(name, choices)

In [71]:
top5_leagues = ['Ligue 1 Uber Eats', 'English Premier Division', 'Italian Serie A',
       'Spanish First Division', 'Bundesliga'] 

In [72]:
url = "https://fbref.com/en/comps/Big5/2022-2023/passing/players/2022-2023-Big-5-European-Leagues-Stats"

In [73]:
data = pd.read_html(url)

In [74]:
data[0].columns = data[0].columns.get_level_values(1)

In [75]:
data[0].head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Cmp,...,Ast,xAG,xA,A-xAG,KP,1/3,PPA,CrsPA,PrgP,Matches
0,1,Brenden Aaronson,us USA,"MF,FW",Leeds United,eng Premier League,21,2000,26.4,592,...,3,4.2,2.6,-1.2,46,47,16,4,86,Matches
1,2,Paxten Aaronson,us USA,"MF,DF",Eint Frankfurt,de Bundesliga,18,2003,1.9,51,...,0,0.0,0.1,0.0,1,3,0,0,6,Matches
2,3,James Abankwah,ie IRL,DF,Udinese,it Serie A,18,2004,0.7,23,...,0,0.0,0.0,0.0,0,0,0,0,0,Matches
3,4,George Abbott,eng ENG,MF,Tottenham,eng Premier League,16,2005,0.0,1,...,0,0.0,0.0,0.0,0,0,0,0,0,Matches
4,5,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,34,1987,37.0,1679,...,2,1.0,0.9,+1.0,13,155,5,0,215,Matches


In [76]:
fbref_players = data[0]["Player"].tolist()
fm_players = fm_df[fm_df.Division.isin(top5_leagues)]["Name"].tolist()

In [77]:
for player_name in fm_players:
    answer, score, other = find_best_match(player_name, fbref_players)
    where_id = fm_df[fm_df.Name==player_name].iloc[0].name
    fm_df.at[where_id, 'fbref_player_name'] = answer

# CSV EXPORT

In [78]:
fm_df.to_csv("players_db/fm23/fm23db_processed.csv", index=False)

In [79]:
# fm_df.query("Name in 'Thiago Silva'").iloc[0].to_dict()